In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Data Loading Process

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn import preprocessing
import gc

'''interaction_data = []
with open("/kaggle/input/goodreads-interactions-poetry/goodreads_interactions_poetry.json", 'r') as f:
    for line in f:
        interaction_data.append(json.loads(line))'''
        
review_data = []
with open("/kaggle/input/goodreads-review-poetry/goodreads_reviews_poetry.json", 'r') as f:
    for line in f:
        review_data.append(json.loads(line))
        
books_data = []
with open("/kaggle/input/goodbooks-books-poetry/goodreads_books_poetry.json", 'r') as f:
    for line in f:
        books_data.append(json.loads(line))
        
        
# interaction_data1 = interaction_data
review_data1 = review_data[:1000000]
book_data1 = books_data[:1000000]



In [ ]:
# Converting into DataFrames

# interaction_df1 = pd.DataFrame(interaction_data1)
review_df1 = pd.DataFrame(review_data1)
book_df1 = pd.DataFrame(book_data1)

## Data Visualization/Cleaning

In [ ]:
review_df1.head(10)

In [ ]:
# interaction_df1.head(10)

In [ ]:
book_df1.head(10)

In [ ]:
# interaction_df1.info()

book_df1.info()

review_df1.info()

In [ ]:
print("Column Names of Interaction Data: {0}",format(list(interaction_df1.columns)))
print("Column Names of Review Data : {0}", format(list(review_df1.columns)))
print("Column Names of Book Data: {0}", format(list(book_df1.columns)))

# To check if there are any null values in the columns

print(interaction_df1.isnull().values.any())
print(review_df1.isnull().values.any())
print(book_df1.isnull().values.any())


# To gauge the unique number of users and books

print(len(interaction_df1.user_id.unique()))
print(len(interaction_df1.book_id.unique()))
print(len(interaction_df1.date_added.unique()))

print(len(review_df1.user_id.unique()))

In [ ]:
# Drop out the irrelevent columns. I'll drop the dates, months and year, alongwith country_code column because they will serve the purpose in implicit recommendation process.
book_df1 = book_df1.drop(columns = ['isbn','series','asin','kindle_asin','description','link','isbn13','edition_information','url','image_url','title','title_without_series','country_code','text_reviews_count','language_code','publication_day','publication_month','is_ebook','format','publication_year'])
# We drop the irrelevant columns from explicit feedback collaborative filtering process.
review_df1 = review_df1.drop(columns = ['review_id','review_text','n_votes','n_comments','date_added','date_updated','read_at','started_at'])
book_df1 = book_df1.drop(columns =['work_id'])


## Data Transformation

In [ ]:
book_df1.book_id = book_df1.book_id.astype(str).astype(int)
book_df1.ratings_count = book_df1.ratings_count.astype(str).astype(int)
# The code says out of memory (at the matrix formation stage) whithout this modification (it would be preferable if we didn't have to place this restriction on the number of ratings)
for i in range(len(book_df1)):
    if book_df1.ratings_count[i] < 50:
        book_df1.drop([i], inplace=True)

review_df1.book_id = review_df1.book_id.astype(str).astype(int)
df = pd.merge(review_df1, book_df1, on='book_id')
df.average_rating = df.average_rating.astype(str).astype(float)
df.ratings_count = df.ratings_count.astype(str).astype(int)
df.user_id = df.user_id.astype(str).astype(str)
df.book_id = df.book_id.astype(str).astype(int)
df['user_idInt']=df['user_id'].str.replace('\D+','').astype(float)
print(len(df.user_idInt.unique()))
df.drop_duplicates(['user_idInt','book_id'])
print(len(df.user_idInt.unique()))
popular_SHELVES = df.popular_shelves
similar_BOOKS = df.similar_books
df = df.drop(columns = ['popular_shelves','similar_books','ratings_count', 'authors'])
df = df.drop(columns = ['average_rating','publisher','num_pages'])



## Train/Test Split

In [ ]:
df_copy = df.copy()
df_train = df_copy.sample(frac=0.8, random_state=0)
df_test = df_copy.drop(df_train.index)

'''print(len(df.user_id.unique()))
users = df['user_idInt'].unique()
books = df['book_id'].unique()

test = pd.DataFrame(columns=df.columns)
train = pd.DataFrame(columns=df.columns)
test_ratio = 0.2 #fraction of data to be used as test set.

for u in users:
    temp = df[df['user_idInt'] == u]
    n = len(temp)
    test_size = int(test_ratio*n)
    

    # temp.drop('index', axis=1, inplace=True)
    
    dummy_test = temp.ix[n-1-test_size :]
    dummy_train = temp.ix[: n-2-test_size]
    
    test = pd.concat([test, dummy_test])
    train = pd.concat([train, dummy_train])

print(test.info())
print(test.describe())'''

## Model

In [ ]:
# Create a books_matrix with the item-user rating values.
books_matrix = df_train.pivot_table(index = 'user_idInt', columns = 'book_id', values = 'rating').fillna(0)
books_matrix.shape
books_matrix.head(10)

mean_rows = []
mean_rows = books_matrix.mean(axis = 1, skipna = True) 

import math
for column in books_matrix.columns:

    books_matrix[column] = books_matrix[column].replace(np.nan, mean_rows[column])




In [ ]:
import sklearn
from sklearn.decomposition import TruncatedSVD

X = books_matrix.values.T
X.shape
#Fitting the Model
SVD = TruncatedSVD(n_components=5, random_state=0)
matrix1 = SVD.fit_transform(X)
matrix1.shape 

X = books_matrix.values
X.shape
#Fitting the Model
SVD = TruncatedSVD(n_components=5, random_state=0)
matrix = SVD.fit_transform(X)
matrix.shape 

svd_out = np.dot(matrix, matrix1.T)

def rmse(true, pred):
    # this will be used towards the end
    x = []
    for i in range(len(true)):
        x.append(true[i] - pred[i])
    return sum([xi*xi for xi in x])/len(x)



''' itemField = formatizer['item']
userField = formatizer['user']


users = list(set(data.ix[:,userField]))
items = list(set(data.ix[:,itemField]))
users_index = {users[i]: i for i in range(len(users))}


itemcols = list(X.columns)
items_index = {itemcols[i]: i for i in range(len(itemcols))}'''




## Evaluation Stage

In [ ]:
pred = [] #to store the predicted ratings
test_cols = [] # to store the ratings of the users that we will actually be testing against
for _,row in df_test.iterrows():
    user = row['user_idInt']
    book = row['book_id']
    if user in books_matrix.index and book in books_matrix.columns:
        u_index = books_matrix.index.index(user)
        i_index = books_matrix.columns.index(book)
        pred_rating = svdout[u_index, i_index]
        test_cols.append(row['ratings'])
    elif user in books_matrix.index and book not in books_matrix.columns:
        u_index = books_matrix.index.index(user)
        pred_rating = np.mean(svdout[u_index, :])
        test_cols.append(row['ratings'])
    else:
        continue
    pred.append(pred_rating)
print(len(test_cols))
print(len(pred))
# print(rmse(test_cols, pred))



In [ ]:
# Following is the method for recommending the books to the user based on their liking as derived from the correlation matrix

#to avoid RuntimeWarning 
#Base class for warnings about dubious runtime behavior.
import warnings
warnings.filterwarnings("ignore",category = RuntimeWarning)

corr = np.corrcoef(matrix1)
print(corr.shape)
print(corr[:10])

#book_ids = books_matrix.columns
title_list = list(books_matrix.columns)

# print(len(title_list))
# title_list = list(book_ids)
#print(title_list)
t = title_list.index(402128)


corr_t = corr[t]
# print(corr_t)
# print(corr_t.shape)
a = []
for i in range(len(corr_t)):
    if corr_t[i]>0.95:
        a.append(title_list[i])

print(a)
    